In [9]:
import glob
import os.path
from xml.dom.minidom import parse as parse_xml, Document

from constants import VINS_MERGED_ANNOTATIONS, VINS_MERGED_YOLO


In [10]:
xmls = glob.glob(VINS_MERGED_ANNOTATIONS + '*.xml')
xmls[0]

'E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/annotations\\100.xml'

'E:/Research/course research projects/data-driven software engineering/datasets/All Dataset/Merged/annotations\\100.xml'

In [11]:
document: Document = parse_xml(xmls[0])

In [12]:
width = document.getElementsByTagName('size')[0].childNodes[1].childNodes[0].nodeValue
height = document.getElementsByTagName('size')[0].childNodes[3].childNodes[0].nodeValue
width = int(width)
height = int(height)

width, height

(540, 897)

(540, 897)

In [13]:
from constants import VINS_CLASSES_COUNT, VINS_CLASSES,reset_classes_counts


def increment_class_count(label: str) -> None:
    if label in VINS_CLASSES_COUNT:
        VINS_CLASSES_COUNT[label] += 1


def encode_class(label: str) -> int:
    if label in VINS_CLASSES:
        return VINS_CLASSES[label]
    print(f"Warning class {label}")
    return -1

In [14]:
objects = document.getElementsByTagName('object')
# for obj in objects:
#     print(obj)

i = 0
tag = objects[i].childNodes[1].childNodes[0].nodeValue
print(tag)
# xmin =
xmin = int(objects[i].childNodes[5].childNodes[1].childNodes[0].nodeValue)
ymin = int(objects[i].childNodes[5].childNodes[3].childNodes[0].nodeValue)
xmax = int(objects[i].childNodes[5].childNodes[5].childNodes[0].nodeValue)
ymax = int(objects[i].childNodes[5].childNodes[7].childNodes[0].nodeValue)

center_x = ((xmax + xmin) / 2) / width
center_y = ((ymax + ymin) / 2) / height
obj_width = (xmax - xmin) / width
obj_height = (ymax - ymin) / height
yolo_obj = f"{encode_class(tag)} {center_x} {center_y} {obj_width} {obj_height}"
# print(encode_class(tag))
print(yolo_obj)

Icon
2 0.9416666666666667 0.07692307692307693 0.06481481481481481 0.044593088071348944
Icon
2 0.9416666666666667 0.07692307692307693 0.06481481481481481 0.044593088071348944


In [19]:
def get_width_height(doc):
    width = doc.getElementsByTagName('size')[0].childNodes[1].childNodes[0].nodeValue
    height = doc.getElementsByTagName('size')[0].childNodes[3].childNodes[0].nodeValue
    width = int(width)
    height = int(height)
    return width, height


def get_object_details_yolo5(object, width, height) -> str:
    tag_idx = None
    for ti in range(len(object.childNodes)):
        if object.childNodes[ti].localName == "name":
            tag_idx = ti
            break

    tag = object.childNodes[tag_idx].childNodes[0].nodeValue
    increment_class_count(tag) # for final report
    bndbox_idx = None
    for node in range(len(object.childNodes)):
      if object.childNodes[node].localName == "bndbox":
          bndbox_idx = node
          break

    xmin_idx = None
    ymin_idx = None
    xmax_idx = None
    ymax_idx = None

    for idx in range(len(object.childNodes[bndbox_idx].childNodes)):
        if object.childNodes[bndbox_idx].childNodes[idx].localName == "xmin":
            xmin_idx = idx
        elif object.childNodes[bndbox_idx].childNodes[idx].localName == "ymin":
            ymin_idx = idx
        elif object.childNodes[bndbox_idx].childNodes[idx].localName == "xmax":
            xmax_idx = idx
        elif object.childNodes[bndbox_idx].childNodes[idx].localName == "ymax":
            ymax_idx = idx

    xmin = int(float(object.childNodes[bndbox_idx].childNodes[xmin_idx].childNodes[0].nodeValue))
    ymin = int(float(object.childNodes[bndbox_idx].childNodes[ymin_idx].childNodes[0].nodeValue))
    xmax = int(float(object.childNodes[bndbox_idx].childNodes[xmax_idx].childNodes[0].nodeValue))
    ymax = int(float(object.childNodes[bndbox_idx].childNodes[ymax_idx].childNodes[0].nodeValue))

    center_x = ((xmax + xmin) / 2) / width
    center_y = ((ymax + ymin) / 2) / height
    obj_width = (xmax - xmin) / width
    obj_height = (ymax - ymin) / height
    encoded_class = encode_class(tag)
    if encoded_class == -1:
        raise Exception("Discarded Class Item")
    yolo_obj = f"{encoded_class} {center_x} {center_y} {obj_width} {obj_height}"
    return yolo_obj

def get_objects_details_yolo5(objects, width, height):
    yolo_objects = []
    for obj in objects:
        try:
            yolo_object = get_object_details_yolo5(obj, width, height)
            yolo_objects.append(yolo_object)
        except Exception as e:
            pass
            # print(str(e))

    return "\n".join(yolo_objects)

def read_xml_doc(filename):
    # returns the document
    return parse_xml(filename)

def transform_doc_to_yolo(filename):
    document = read_xml_doc(filename)
    width, height = get_width_height(document)
    objects = document.getElementsByTagName('object')
    return get_objects_details_yolo5(objects,width,height)

def transform_doc_to_yolo_file(filename, path):
    with open(path,'w') as file:
        print(transform_doc_to_yolo(filename), file=file)

def get_filename(full_filename):
    return full_filename.replace("\\","/").split("/")[-1]

def transform_docs_to_yolo_files():
    xmls = glob.glob(VINS_MERGED_ANNOTATIONS + '*.xml')
    for xml in xmls:
        # print(xml)
        transform_doc_to_yolo_file(xml,f"{VINS_MERGED_YOLO}{get_filename(xml.split('.')[0])}.txt")


    print("DONE!")
    print(VINS_CLASSES_COUNT)

reset_classes_counts()
transform_docs_to_yolo_files()

Warning class Toolbar
Warning class Bottom_Navigation
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Bottom_Navigation
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Bottom_Navigation
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Bottom_Navigation
Warning class Multi_Tab
Warning class Bottom_Navigation
Warning class Bottom_Navigation
Warning class Bottom_Navigation
Warning class Multi_Tab
Warning class Bottom_Navigation
Warning class Bottom_Navigation
Warning class Multi_Tab
Warning class Multi_Tab
Warning class Bottom_Navigation
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Toolbar
Warning class Multi_Tab
Warning class Bottom_Navigation
Warning class Bottom_Navigation
Warning class Bottom_Navigation
Warning class Multi_Tab
Warning class Toolbar


In [18]:

# print(VINS_CLASSES_COUNT)
# reset_classes_counts()

{'BackgroundImage': 0, 'CheckedTextView': 0, 'Icon': 0, 'EditText': 0, 'Image': 0, 'Text': 0, 'TextButton': 0, 'Drawer': 0, 'PageIndicator': 0, 'UpperTaskBar': 0, 'Modal': 0, 'Switch': 0}
